In [1]:
import numpy as np
import sklearn
import nltk

from generateWordFrequency import *
from naiveBayes import *
from fileWriteFunctions import *

In [2]:
df_2018 = getDataframe(2018)
df_2018 = addTokenizedColumnofTitle(df_2018)

In [3]:
df_2018.head()

,Title,Post Type,Number of Comments,Points,Author,tokenized_title
0,Top Hacker News Books in 2017,story,0.0,1,0x54MUR41,"[top, hacker, news, book, in, 2017]"
1,Beijing enjoys best winter air quality in five...,story,0.0,2,gpetukhov,"[beijing, enjoy, best, winter, air, quality, i..."
2,Ask HN: Any domain name registrars that don't ...,ask_hn,2.0,1,glockenspielen,"[ask, hn, any, domain, name, registrars, that,..."
3,Controversial Therapy Has Led to Death Threats...,story,0.0,2,cpncrunch,"[controversial, therapy, have, lead, to, death..."
4,Ruby 3x3 – Ruby 3 Will Be 3 Times Faster – Wha...,story,0.0,2,geraldbauer,"[ruby, 3x3, ruby, 3, will, be, 3, time, faster..."


In [4]:
df_2018['Title'][2]

"Ask HN: Any domain name registrars that don't require JavaScript?"

In [5]:
AllClasses = list(np.unique(df_2018['Post Type']))
AllClasses = ['story', 'ask_hn', 'show_hn', 'poll']
delta = 0.5
appendClassPrefix = 'prob_'

## Calculating frequency of each word and given their conditional post type

In [6]:
words_2018 = getWordFrequencyDataframe(df_2018,AllClasses)
totalNumberOfWords = getTotalWordCount(words_2018)
words_2018 = obtainDataframeWithClassProbabilities(words_2018, AllClasses, delta, appendClassPrefix)

In [7]:
priorProbabilities = {}
unique, counts = np.unique(df_2018['Post Type'], return_counts=True)
for index in range(len(unique)):
    priorProbabilities[unique[index]] = counts[index]/np.sum(counts)
print(priorProbabilities)

{'ask_hn': 0.04516194251591264, 'poll': 9.025889862481542e-05, 'show_hn': 0.036915889537549505, 'story': 0.917831909047913}


In [8]:
writeModel(words_2018,'model-2018.txt',AllClasses,appendClassPrefix)

In [9]:
words_2018 = renameModelRows(words_2018, AllClasses, appendClassPrefix)

In [10]:
model = [words_2018, priorProbabilities]

In [11]:
sent = "gg fgf fgdfh"
getRandomSentencePrediction(sent,model)

'story'

In [12]:
words_2018.index.values

array(['story', 'ask_hn', 'show_hn', 'poll'], dtype=object)

In [13]:
words_2018.head(10)

,top,hacker,news,book,in,2017,beijing,enjoy,best,winter,...,400gb,shareseer,öyster,face-detecting,rainer,rilke,computing.foundation,integer32,nichols,goulding
story,0.000601,0.000493,0.000670,0.000604,0.015977,0.000984,0.000038,0.000023,0.000807,0.000069,...,7.055676e-07,7.055676e-07,7.055676e-07,7.055676e-07,7.055676e-07,7.055676e-07,7.055676e-07,7.055676e-07,7.055676e-07,0.000001
ask_hn,0.000214,0.000493,0.000635,0.001421,0.008090,0.000128,0.000009,0.000060,0.003693,0.000014,...,2.847721e-06,2.847721e-06,2.847721e-06,2.847721e-06,2.847721e-06,2.847721e-06,2.847721e-06,2.847721e-06,2.847721e-06,0.000003
show_hn,0.000478,0.000701,0.001287,0.000540,0.008017,0.000045,0.000003,0.000010,0.000401,0.000017,...,3.487066e-06,3.487066e-06,3.487066e-06,3.487066e-06,3.487066e-06,3.487066e-06,3.487066e-06,3.487066e-06,3.487066e-06,0.000003
poll,0.000011,0.000011,0.000011,0.000032,0.000032,0.000011,0.000011,0.000011,0.000032,0.000011,...,1.074576e-05,1.074576e-05,1.074576e-05,1.074576e-05,1.074576e-05,1.074576e-05,1.074576e-05,1.074576e-05,1.074576e-05,0.000011


## PART 2

In [ ]:
testData = getDataframe(2019)
testData = addTokenizedColumnofTitle(testData)
testData = testData.reset_index()

In [ ]:
testData = generateCondClassProb(testData, model)
testData.head()

In [ ]:
testData['predicted'] = generatePrediction(testData,AllClasses)
testData.head()   

In [ ]:
testResults = comparePredictions(testData,AllClasses)
testResults.head()

In [ ]:
writeDataframe(testResults,'result.txt')

In [ ]:
check = testResults.comparision.value_counts()
accuracy = check[True]/(check[True]+check[False])
print(accuracy)

In [ ]:
df_confusion = pd.crosstab(testResults['Post Type'], testResults['predicted'])
df_confusion['poll'] = 0
print(df_confusion)

In [ ]:
df_confusion['precision'] = 0.0
df_confusion['recall'] = 0.0
#df['F1_score'] = 0.0
transpose_matrix = df_confusion.transpose()
for i in transpose_matrix.columns:
    if np.sum(df_confusion[i])==0:
        total = 1
    else:
        total = np.sum(df_confusion[i])
    df_confusion['precision'][i] = df_confusion[i][i]/total
    df_confusion['recall'][i] = df_confusion[i][i]/np.sum(transpose_matrix[i])
df_confusion['F1_score'] = 2/(1/df_confusion['precision']+1/df_confusion['recall'])
print(df_confusion)